In [1]:
import os
import numpy as np 
import torch 
from PIL import Image
import math
from random import randint
import torchvision.transforms as transforms
import pandas as pd

class seti(torch.utils.data.Dataset):
    def __init__(self, img_path, trainortest='train', label_path=None, aug=None, cfg=None):
        self.img_path = img_path
        self.trainortest = trainortest
        if trainortest is "test":
            self.labels = pd.read_csv(label_path)
        else: 
            self.labels = pd.read_csv(img_path)
        self.path_labels = [self.img_path + "train/" f"{ids[0]}/{ids}.npy" for ids in self.labels["id"].values]
        self.aims_labels = [ids for ids in self.labels["target"].values.astype("f")]
        self.aug = aug

    def __getitem__(self, index):
        """Return transformed image and label for given index."""
        path = self.paths[index]
        img = self._read_cadence_array(path)
        if self.aug is not None:
            img = self.aug(img.astype(np.uint8))
        img = img.float()
        if self.labels is not None:
            label = self.labels[index]
            return img, label 
        return img, f"{self.paths[index][-19:-4]}"

    def _read_cadence_array(self, path):
        """Read cadence file and reshape"""
        img = np.load(path)[[0, 2, 4]] 
        img = np.vstack(img)
        img = img.transpose(1, 0)  
        img = img.astype("f")[..., np.newaxis]  
        return img
    
    def __len__(self):
        """Return num of cadence snippets"""
        return len(self.paths)

    def _resize(self, img_data):
        large_edge = max(img_data.shape[0], img_data.shape[1])
        resize_factor = self.img_size/large_edge
        zero_data = np.zeros((self.img_size, self.img_size,3))
        img_data = cv2.resize(img_data, (min(math.ceil(img_data.shape[1]*resize_factor),self.img_size),
            min(math.ceil(img_data.shape[0]*resize_factor),self.img_size)))
        if img_data.shape[0] >= img_data.shape[1]:
            gap = (self.img_size-img_data.shape[1])//2
            zero_data[:,gap:img_data.shape[1]+gap,:] = img_data
        else:
            gap = (self.img_size-img_data.shape[0])//2
            zero_data[gap:img_data.shape[0]+gap,:,:] = img_data
        return zero_data


In [2]:
import numpy as np
import torchvision.transforms as transforms
import torch
import cv2
#from . import seti

# LOADER_LUT = {
#         'waixingren': waixingren.seti,
#     }


def get_loader(dataset_type, data_path, loader_type, label_path=None, cfg=None, logger=None):
    if loader_type == 'train':
        if cfg.USE_AUG == True:
            train_aug = transforms.Compose([
                transforms.ToPILImage(),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(cfg.ROATION),
                #transforms.RandomCrop(cfg.CROP, cfg.PAD),
                transforms.RandomResizedCrop(cfg.CROP),
                transforms.ToTensor(),
                #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])
        else:
            train_aug = None

        try: 
            _data_class = LOADER_LUT.get(dataset_type)    
        except:
            logger.error("dataset type error, {} not exist".format(dataset_type))

        _data = _data_class(data_path,  dtype='train', label_path=label_path, aug=train_aug, cfg=cfg.RESIZE) 
        data_loader = torch.utils.data.DataLoader(_data,
            batch_size=cfg.BATCHSIZE, shuffle=True, num_workers=cfg.NUM_WORKERS,
            drop_last=False)

    elif loader_type == 'eval':
        if cfg.USE_AUG == True:
            val_aug = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize(int(cfg.CROP/0.875)),
                transforms.CenterCrop(cfg.CROP),
                transforms.ToTensor(),
                #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])
        else:
            val_aug = None
        try: 
            _data_class = LOADER_LUT.get(dataset_type)    
        except:
            logger.error("dataset type error, {} not exist".format(dataset_type))

        _data = _data_class(data_path,  dtype='eval', label_path=label_path, aug=val_aug, cfg=cfg.RESIZE) 
        data_loader = torch.utils.data.DataLoader(_data,
            batch_size=cfg.BATCHSIZE, shuffle=False, num_workers=cfg.NUM_WORKERS,
            drop_last=False)

    elif loader_type == 'self_test':
        augmentation = transforms.Compose([
                transforms.ToTensor(),
                #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])
        try: 
            _data_class = LOADER_LUT.get(dataset_type)    
        except:
            logger.error("dataset type error, {} not exist".format(dataset_type))
        _data = _data_class(data_path,  dtype='train', label_path=label_path, aug=augmentation, cfg=cfg) 
        data_loader = torch.utils.data.DataLoader(_data,
        batch_size=2, shuffle=True, num_workers=0,
        drop_last=True)
        
    elif loader_type == 'test':
        augmentaiton = transforms.Compose([
            transforms.ToTensor()
        ])
        #augmentaiton = None
        try: 
            _data_class = LOADER_LUT.get(dataset_type)    
        except:
            logger.error("dataset type error, {} not exist".format(dataset_type))
        _data = _data_class(data_path, aug=augmentaiton, test_data=True)
        data_loader = torch.utils.data.DataLoader(_data,
        batch_size=1, shuffle=False, num_workers=0,
        drop_last=False)

    else:
        logger.error("error, only support train type dataloader")
    
    return data_loader

In [3]:

import torch
import torch.nn as nn



class EffHead(nn.Module):
    """EfficientNet head: 1x1, BN, Swish, AvgPool, Dropout, FC."""

    def __init__(self, w_in, w_out, nc):
        super(EffHead, self).__init__()
        self.conv = nn.Conv2d(w_in, w_out, 1, stride=1, padding=0, bias=False)
        self.conv_bn = nn.BatchNorm2d(w_out, eps=1e-5, momentum=0.1)
        self.conv_swish = Swish()
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        if 0 > 0.0:
            self.dropout = nn.Dropout(0)
        self.fc = nn.Linear(w_out, nc, bias=True)

    def forward(self, x):
        x = self.conv_swish(self.conv_bn(self.conv(x)))
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x) if hasattr(self, "dropout") else x
        x = self.fc(x)
        return x

    # @staticmethod
    # def complexity(cx, w_in, w_out, nc):
    #     cx = net.complexity_conv2d(cx, w_in, w_out, 1, 1, 0)
    #     cx = net.complexity_batchnorm2d(cx, w_out)
    #     cx["h"], cx["w"] = 1, 1
    #     cx = net.complexity_conv2d(cx, w_out, nc, 1, 1, 0, bias=True)
    #     return cx


class Swish(nn.Module):
    """Swish activation function: x * sigmoid(x)."""

    def __init__(self):
        super(Swish, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)


class SE(nn.Module):
    """Squeeze-and-Excitation (SE) block w/ Swish: AvgPool, FC, Swish, FC, Sigmoid."""

    def __init__(self, w_in, w_se):
        super(SE, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.f_ex = nn.Sequential(
            nn.Conv2d(w_in, w_se, 1, bias=True),
            Swish(),
            nn.Conv2d(w_se, w_in, 1, bias=True),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.f_ex(self.avg_pool(x))

    # @staticmethod
    # def complexity(cx, w_in, w_se):
    #     h, w = cx["h"], cx["w"]
    #     cx["h"], cx["w"] = 1, 1
    #     cx = net.complexity_conv2d(cx, w_in, w_se, 1, 1, 0, bias=True)
    #     cx = net.complexity_conv2d(cx, w_se, w_in, 1, 1, 0, bias=True)
    #     cx["h"], cx["w"] = h, w
    #     return cx


class MBConv(nn.Module):
    """Mobile inverted bottleneck block w/ SE (MBConv)."""

    def __init__(self, w_in, exp_r, kernel, stride, se_r, w_out):
        # expansion, 3x3 dwise, BN, Swish, SE, 1x1, BN, skip_connection
        super(MBConv, self).__init__()
        self.exp = None
        w_exp = int(w_in * exp_r)
        if w_exp != w_in:
            self.exp = nn.Conv2d(w_in, w_exp, 1, stride=1, padding=0, bias=False)
            self.exp_bn = nn.BatchNorm2d(w_exp, eps=1e-5, momentum=0.1)
            self.exp_swish = Swish()
        dwise_args = {"groups": w_exp, "padding": (kernel - 1) // 2, "bias": False}
        self.dwise = nn.Conv2d(w_exp, w_exp, kernel, stride=stride, **dwise_args)
        self.dwise_bn = nn.BatchNorm2d(w_exp, eps=1e-5, momentum=0.1)
        self.dwise_swish = Swish()
        self.se = SE(w_exp, int(w_in * se_r))
        self.lin_proj = nn.Conv2d(w_exp, w_out, 1, stride=1, padding=0, bias=False)
        self.lin_proj_bn = nn.BatchNorm2d(w_out, eps=1e-5, momentum=0.1)
        # Skip connection if in and out shapes are the same (MN-V2 style)
        self.has_skip = stride == 1 and w_in == w_out

    def forward(self, x):
        f_x = x
        if self.exp:
            f_x = self.exp_swish(self.exp_bn(self.exp(f_x)))
        f_x = self.dwise_swish(self.dwise_bn(self.dwise(f_x)))
        f_x = self.se(f_x)
        f_x = self.lin_proj_bn(self.lin_proj(f_x))
        if self.has_skip:
            f_x = x + f_x
        return f_x

    # @staticmethod
    # def complexity(cx, w_in, exp_r, kernel, stride, se_r, w_out):
    #     w_exp = int(w_in * exp_r)
    #     if w_exp != w_in:
    #         cx = net.complexity_conv2d(cx, w_in, w_exp, 1, 1, 0)
    #         cx = net.complexity_batchnorm2d(cx, w_exp)
    #     padding = (kernel - 1) // 2
    #     cx = net.complexity_conv2d(cx, w_exp, w_exp, kernel, stride, padding, w_exp)
    #     cx = net.complexity_batchnorm2d(cx, w_exp)
    #     cx = SE.complexity(cx, w_exp, int(w_in * se_r))
    #     cx = net.complexity_conv2d(cx, w_exp, w_out, 1, 1, 0)
    #     cx = net.complexity_batchnorm2d(cx, w_out)
    #    return cx


class EffStage(nn.Module):
    """EfficientNet stage."""

    def __init__(self, w_in, exp_r, kernel, stride, se_r, w_out, d):
        super(EffStage, self).__init__()
        for i in range(d):
            b_stride = stride if i == 0 else 1
            b_w_in = w_in if i == 0 else w_out
            name = "b{}".format(i + 1)
            self.add_module(name, MBConv(b_w_in, exp_r, kernel, b_stride, se_r, w_out))

    def forward(self, x):
        for block in self.children():
            x = block(x)
        return x

    # @staticmethod
    # def complexity(cx, w_in, exp_r, kernel, stride, se_r, w_out, d):
    #     for i in range(d):
    #         b_stride = stride if i == 0 else 1
    #         b_w_in = w_in if i == 0 else w_out
    #         cx = MBConv.complexity(cx, b_w_in, exp_r, kernel, b_stride, se_r, w_out)
    #     return cx


class StemIN(nn.Module):
    """EfficientNet stem for ImageNet: 3x3, BN, Swish."""

    def __init__(self, w_in, w_out):
        super(StemIN, self).__init__()
        self.conv = nn.Conv2d(w_in, w_out, 3, stride=2, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(w_out, eps=1e-5, momentum=0.1)
        self.swish = Swish()

    def forward(self, x):
        for layer in self.children():
            x = layer(x)
        return x

    # @staticmethod
    # def complexity(cx, w_in, w_out):
    #     cx = net.complexity_conv2d(cx, w_in, w_out, 3, 2, 1)
    #     cx = net.complexity_batchnorm2d(cx, w_out)
    #     return cx


class EffNet(nn.Module):
    """EfficientNet model."""

    @staticmethod
    def get_args(cfg):
        return {
            "stem_w": cfg.EFFNET.STEM_W,
            "ds": cfg.EFFNET.DEPTHS,
            "ws": cfg.EFFNET.WIDTHS,
            "exp_rs": cfg.EFFNET.EXP_RATIOS,
            "se_r": cfg.EFFNET.SE_R,
            "ss": cfg.EFFNET.STRIDES,
            "ks": cfg.EFFNET.KERNELS,
            "head_w": cfg.EFFNET.HEAD_W,
            "nc": cfg.CLASS_NUM,
        }

    def __init__(self,cfg,logger):
        super(EffNet, self).__init__()
        self._construct(**EffNet.get_args(cfg))

    def _construct(self, stem_w, ds, ws, exp_rs, se_r, ss, ks, head_w, nc):
        stage_params = list(zip(ds, ws, exp_rs, ss, ks))
        self.stem = StemIN(3, stem_w)
        prev_w = stem_w
        for i, (d, w, exp_r, stride, kernel) in enumerate(stage_params):
            name = "s{}".format(i + 1)
            self.add_module(name, EffStage(prev_w, exp_r, kernel, stride, se_r, w, d))
            prev_w = w
        self.head = EffHead(prev_w, head_w, nc)

    def forward(self, x):
        for module in self.children():
            x = module(x)
        return x

    # @staticmethod
    # def complexity(cx):
    #     """Computes model complexity. If you alter the model, make sure to update."""
    #     return EffNet._complexity(cx, **EffNet.get_args())

    # @staticmethod
    # def _complexity(cx, stem_w, ds, ws, exp_rs, se_r, ss, ks, head_w, nc):
    #     stage_params = list(zip(ds, ws, exp_rs, ss, ks))
    #     cx = StemIN.complexity(cx, 3, stem_w)
    #     prev_w = stem_w
    #     for d, w, exp_r, stride, kernel in stage_params:
    #         cx = EffStage.complexity(cx, prev_w, exp_r, kernel, stride, se_r, w, d)
    #         prev_w = w
    #     cx = EffHead.complexity(cx, prev_w, head_w, nc)
    #     return cx

In [4]:
#import pycls.core.net as net
import torch.nn as nn
import numpy as np

def get_stem_fun(stem_type):
    """Retrieves the stem function by name."""
    stem_funs = {
        "res_stem_cifar": ResStemCifar,
        "res_stem_in": ResStemIN,
        "simple_stem_in": SimpleStemIN,
    }
    err_str = "Stem type '{}' not supported"
    assert stem_type in stem_funs.keys(), err_str.format(stem_type)
    return stem_funs[stem_type]


def get_block_fun(block_type):
    """Retrieves the block function by name."""
    block_funs = {
        "vanilla_block": VanillaBlock,
        "res_basic_block": ResBasicBlock,
        "res_bottleneck_block": ResBottleneckBlock,
    }
    err_str = "Block type '{}' not supported"
    assert block_type in block_funs.keys(), err_str.format(block_type)
    return block_funs[block_type]


class AnyHead(nn.Module):
    """AnyNet head: AvgPool, 1x1."""

    def __init__(self, w_in, nc):
        super(AnyHead, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(w_in, nc, bias=True)

    def forward(self, x):
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    # @staticmethod
    # def complexity(cx, w_in, nc):
    #     cx["h"], cx["w"] = 1, 1
    #     cx = net.complexity_conv2d(cx, w_in, nc, 1, 1, 0, bias=True)
    #     return cx


class VanillaBlock(nn.Module):
    """Vanilla block: [3x3 conv, BN, Relu] x2."""

    def __init__(self, w_in, w_out, stride, bm=None, gw=None, se_r=None):
        err_str = "Vanilla block does not support bm, gw, and se_r options"
        assert bm is None and gw is None and se_r is None, err_str
        super(VanillaBlock, self).__init__()
        self.a = nn.Conv2d(w_in, w_out, 3, stride=stride, padding=1, bias=False)
        self.a_bn = nn.BatchNorm2d(w_out, eps=1e-5, momentum=0.1)
        self.a_relu = nn.ReLU(inplace=True)
        self.b = nn.Conv2d(w_out, w_out, 3, stride=1, padding=1, bias=False)
        self.b_bn = nn.BatchNorm2d(w_out, eps=1e-5, momentum=0.1)
        self.b_relu = nn.ReLU(inplace=True)

    def forward(self, x):
        for layer in self.children():
            x = layer(x)
        return x

    # @staticmethod
    # def complexity(cx, w_in, w_out, stride, bm=None, gw=None, se_r=None):
    #     err_str = "Vanilla block does not support bm, gw, and se_r options"
    #     assert bm is None and gw is None and se_r is None, err_str
    #     cx = net.complexity_conv2d(cx, w_in, w_out, 3, stride, 1)
    #     cx = net.complexity_batchnorm2d(cx, w_out)
    #     cx = net.complexity_conv2d(cx, w_out, w_out, 3, 1, 1)
    #     cx = net.complexity_batchnorm2d(cx, w_out)
    #     return cx


class BasicTransform(nn.Module):
    """Basic transformation: [3x3 conv, BN, Relu] x2."""

    def __init__(self, w_in, w_out, stride):
        super(BasicTransform, self).__init__()
        self.a = nn.Conv2d(w_in, w_out, 3, stride=stride, padding=1, bias=False)
        self.a_bn = nn.BatchNorm2d(w_out, eps=1e-5, momentum=0.1)
        self.a_relu = nn.ReLU(inplace=True)
        self.b = nn.Conv2d(w_out, w_out, 3, stride=1, padding=1, bias=False)
        self.b_bn = nn.BatchNorm2d(w_out, eps=1e-5, momentum=0.1)
        self.b_bn.final_bn = True

    def forward(self, x):
        for layer in self.children():
            x = layer(x)
        return x

    # @staticmethod
    # def complexity(cx, w_in, w_out, stride):
    #     cx = net.complexity_conv2d(cx, w_in, w_out, 3, stride, 1)
    #     cx = net.complexity_batchnorm2d(cx, w_out)
    #     cx = net.complexity_conv2d(cx, w_out, w_out, 3, 1, 1)
    #     cx = net.complexity_batchnorm2d(cx, w_out)
    #     return cx


class ResBasicBlock(nn.Module):
    """Residual basic block: x + F(x), F = basic transform."""

    def __init__(self, w_in, w_out, stride, bm=None, gw=None, se_r=None):
        err_str = "Basic transform does not support bm, gw, and se_r options"
        assert bm is None and gw is None and se_r is None, err_str
        super(ResBasicBlock, self).__init__()
        self.proj_block = (w_in != w_out) or (stride != 1)
        if self.proj_block:
            self.proj = nn.Conv2d(w_in, w_out, 1, stride=stride, padding=0, bias=False)
            self.bn = nn.BatchNorm2d(w_out, eps=1e-5, momentum=0.1)
        self.f = BasicTransform(w_in, w_out, stride)
        self.relu = nn.ReLU(True)

    def forward(self, x):
        if self.proj_block:
            x = self.bn(self.proj(x)) + self.f(x)
        else:
            x = x + self.f(x)
        x = self.relu(x)
        return x

    # @staticmethod
    # def complexity(cx, w_in, w_out, stride, bm=None, gw=None, se_r=None):
    #     err_str = "Basic transform does not support bm, gw, and se_r options"
    #     assert bm is None and gw is None and se_r is None, err_str
    #     proj_block = (w_in != w_out) or (stride != 1)
    #     if proj_block:
    #         h, w = cx["h"], cx["w"]
    #         cx = net.complexity_conv2d(cx, w_in, w_out, 1, stride, 0)
    #         cx = net.complexity_batchnorm2d(cx, w_out)
    #         cx["h"], cx["w"] = h, w  # parallel branch
    #     cx = BasicTransform.complexity(cx, w_in, w_out, stride)
    #     return cx


class SE(nn.Module):
    """Squeeze-and-Excitation (SE) block: AvgPool, FC, ReLU, FC, Sigmoid."""

    def __init__(self, w_in, w_se):
        super(SE, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.f_ex = nn.Sequential(
            nn.Conv2d(w_in, w_se, 1, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(w_se, w_in, 1, bias=True),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.f_ex(self.avg_pool(x))

    # @staticmethod
    # def complexity(cx, w_in, w_se):
    #     h, w = cx["h"], cx["w"]
    #     cx["h"], cx["w"] = 1, 1
    #     cx = net.complexity_conv2d(cx, w_in, w_se, 1, 1, 0, bias=True)
    #     cx = net.complexity_conv2d(cx, w_se, w_in, 1, 1, 0, bias=True)
    #     cx["h"], cx["w"] = h, w
    #     return cx


class BottleneckTransform(nn.Module):
    """Bottleneck transformation: 1x1, 3x3 [+SE], 1x1."""

    def __init__(self, w_in, w_out, stride, bm, gw, se_r):
        super(BottleneckTransform, self).__init__()
        w_b = int(round(w_out * bm))
        g = w_b // gw
        self.a = nn.Conv2d(w_in, w_b, 1, stride=1, padding=0, bias=False)
        self.a_bn = nn.BatchNorm2d(w_b, eps=1e-5, momentum=0.1)
        self.a_relu = nn.ReLU(inplace=True)
        self.b = nn.Conv2d(w_b, w_b, 3, stride=stride, padding=1, groups=g, bias=False)
        self.b_bn = nn.BatchNorm2d(w_b, eps=1e-5, momentum=0.1)
        self.b_relu = nn.ReLU(inplace=True)
        if se_r:
            w_se = int(round(w_in * se_r))
            self.se = SE(w_b, w_se)
        self.c = nn.Conv2d(w_b, w_out, 1, stride=1, padding=0, bias=False)
        self.c_bn = nn.BatchNorm2d(w_out, eps=1e-5, momentum=0.1)
        self.c_bn.final_bn = True

    def forward(self, x):
        for layer in self.children():
            x = layer(x)
        return x

    # @staticmethod
    # def complexity(cx, w_in, w_out, stride, bm, gw, se_r):
    #     w_b = int(round(w_out * bm))
    #     g = w_b // gw
    #     cx = net.complexity_conv2d(cx, w_in, w_b, 1, 1, 0)
    #     cx = net.complexity_batchnorm2d(cx, w_b)
    #     cx = net.complexity_conv2d(cx, w_b, w_b, 3, stride, 1, g)
    #     cx = net.complexity_batchnorm2d(cx, w_b)
    #     if se_r:
    #         w_se = int(round(w_in * se_r))
    #         cx = SE.complexity(cx, w_b, w_se)
    #     cx = net.complexity_conv2d(cx, w_b, w_out, 1, 1, 0)
    #     cx = net.complexity_batchnorm2d(cx, w_out)
    #     return cx


class ResBottleneckBlock(nn.Module):
    """Residual bottleneck block: x + F(x), F = bottleneck transform."""

    def __init__(self, w_in, w_out, stride, bm=1.0, gw=1, se_r=None):
        super(ResBottleneckBlock, self).__init__()
        # Use skip connection with projection if shape changes
        self.proj_block = (w_in != w_out) or (stride != 1)
        if self.proj_block:
            self.proj = nn.Conv2d(w_in, w_out, 1, stride=stride, padding=0, bias=False)
            self.bn = nn.BatchNorm2d(w_out, eps=1e-5, momentum=0.1)
        self.f = BottleneckTransform(w_in, w_out, stride, bm, gw, se_r)
        self.relu = nn.ReLU(True)

    def forward(self, x):
        if self.proj_block:
            x = self.bn(self.proj(x)) + self.f(x)
        else:
            x = x + self.f(x)
        x = self.relu(x)
        return x

    # @staticmethod
    # def complexity(cx, w_in, w_out, stride, bm=1.0, gw=1, se_r=None):
    #     proj_block = (w_in != w_out) or (stride != 1)
    #     if proj_block:
    #         h, w = cx["h"], cx["w"]
    #         cx = net.complexity_conv2d(cx, w_in, w_out, 1, stride, 0)
    #         cx = net.complexity_batchnorm2d(cx, w_out)
    #         cx["h"], cx["w"] = h, w  # parallel branch
    #     cx = BottleneckTransform.complexity(cx, w_in, w_out, stride, bm, gw, se_r)
    #     return cx


class ResStemCifar(nn.Module):
    """ResNet stem for CIFAR: 3x3, BN, ReLU."""

    def __init__(self, w_in, w_out):
        super(ResStemCifar, self).__init__()
        self.conv = nn.Conv2d(w_in, w_out, 3, stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(w_out, eps=1e-5, momentum=0.1)
        self.relu = nn.ReLU(True)

    def forward(self, x):
        for layer in self.children():
            x = layer(x)
        return x

    # @staticmethod
    # def complexity(cx, w_in, w_out):
    #     cx = net.complexity_conv2d(cx, w_in, w_out, 3, 1, 1)
    #     cx = net.complexity_batchnorm2d(cx, w_out)
    #     return cx


class ResStemIN(nn.Module):
    """ResNet stem for ImageNet: 7x7, BN, ReLU, MaxPool."""

    def __init__(self, w_in, w_out):
        super(ResStemIN, self).__init__()
        self.conv = nn.Conv2d(w_in, w_out, 7, stride=2, padding=3, bias=False)
        self.bn = nn.BatchNorm2d(w_out, eps=1e-5, momentum=0.1)
        self.relu = nn.ReLU(True)
        self.pool = nn.MaxPool2d(3, stride=2, padding=1)

    def forward(self, x):
        for layer in self.children():
            x = layer(x)
        return x

    # @staticmethod
    # def complexity(cx, w_in, w_out):
    #     cx = net.complexity_conv2d(cx, w_in, w_out, 7, 2, 3)
    #     cx = net.complexity_batchnorm2d(cx, w_out)
    #     cx = net.complexity_maxpool2d(cx, 3, 2, 1)
    #     return cx


class SimpleStemIN(nn.Module):
    """Simple stem for ImageNet: 3x3, BN, ReLU."""

    def __init__(self, w_in, w_out):
        super(SimpleStemIN, self).__init__()
        self.conv = nn.Conv2d(w_in, w_out, 3, stride=2, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(w_out, eps=1e-5, momentum=0.1)
        self.relu = nn.ReLU(True)

    def forward(self, x):
        for layer in self.children():
            x = layer(x)
        return x

    # @staticmethod
    # def complexity(cx, w_in, w_out):
    #     cx = net.complexity_conv2d(cx, w_in, w_out, 3, 2, 1)
    #     cx = net.complexity_batchnorm2d(cx, w_out)
    #     return cx


class AnyStage(nn.Module):
    """AnyNet stage (sequence of blocks w/ the same output shape)."""

    def __init__(self, w_in, w_out, stride, d, block_fun, bm, gw, se_r):
        super(AnyStage, self).__init__()
        for i in range(d):
            b_stride = stride if i == 0 else 1
            b_w_in = w_in if i == 0 else w_out
            name = "b{}".format(i + 1)
            self.add_module(name, block_fun(b_w_in, w_out, b_stride, bm, gw, se_r))

    def forward(self, x):
        for block in self.children():
            x = block(x)
        return x

    # @staticmethod
    # def complexity(cx, w_in, w_out, stride, d, block_fun, bm, gw, se_r):
    #     for i in range(d):
    #         b_stride = stride if i == 0 else 1
    #         b_w_in = w_in if i == 0 else w_out
    #         cx = block_fun.complexity(cx, b_w_in, w_out, b_stride, bm, gw, se_r)
    #     return cx


class AnyNet(nn.Module):
    """AnyNet model."""

    @staticmethod
    def get_args(cfg):
        return {
            "stem_type": cfg.ANYNET.STEM_TYPE,
            "stem_w": cfg.ANYNET.STEM_W,
            "block_type": cfg.ANYNET.BLOCK_TYPE,
            "ds": cfg.ANYNET.DEPTHS,
            "ws": cfg.ANYNET.WIDTHS,
            "ss": cfg.ANYNET.STRIDES,
            "bms": cfg.ANYNET.BOT_MULS,
            "gws": cfg.ANYNET.GROUP_WS,
            "se_r": cfg.ANYNET.SE_R if cfg.ANYNET.SE_ON else None,
            "nc": cfg.CLASS_NUM,
        }

    def __init__(self, cfg, logger, **kwargs):
        super(AnyNet, self).__init__()
        kwargs = self.get_args(cfg) if not kwargs else kwargs
        self._construct(**kwargs)
        #self.apply(net.init_weights)

    def _construct(self, stem_type, stem_w, block_type, ds, ws, ss, bms, gws, se_r, nc):
        # Generate dummy bot muls and gs for models that do not use them
        bms = bms if bms else [None for _d in ds]
        gws = gws if gws else [None for _d in ds]
        stage_params = list(zip(ds, ws, ss, bms, gws))
        stem_fun = get_stem_fun(stem_type)
        self.stem = stem_fun(3, stem_w)
        block_fun = get_block_fun(block_type)
        prev_w = stem_w
        for i, (d, w, s, bm, gw) in enumerate(stage_params):
            name = "s{}".format(i + 1)
            self.add_module(name, AnyStage(prev_w, w, s, d, block_fun, bm, gw, se_r))
            prev_w = w
        self.head = AnyHead(w_in=prev_w, nc=nc)

    def forward(self, x):
        for module in self.children():
            x = module(x)
        return x

    # @staticmethod
    # def complexity(cx, **kwargs):
    #     """Computes model complexity. If you alter the model, make sure to update."""
    #     kwargs = AnyNet.get_args() if not kwargs else kwargs
    #     return AnyNet._complexity(cx, **kwargs)

    # @staticmethod
    # def _complexity(cx, stem_type, stem_w, block_type, ds, ws, ss, bms, gws, se_r, nc):
    #     bms = bms if bms else [None for _d in ds]
    #     gws = gws if gws else [None for _d in ds]
    #     stage_params = list(zip(ds, ws, ss, bms, gws))
    #     stem_fun = get_stem_fun(stem_type)
    #     cx = stem_fun.complexity(cx, 3, stem_w)
    #     block_fun = get_block_fun(block_type)
    #     prev_w = stem_w
    #     for d, w, s, bm, gw in stage_params:
    #         cx = AnyStage.complexity(cx, prev_w, w, s, d, block_fun, bm, gw, se_r)
    #         prev_w = w
    #     cx = AnyHead.complexity(cx, prev_w, nc)
    #     return cx


def quantize_float(f, q):
    """Converts a float to closest non-zero int divisible by q."""
    return int(round(f / q) * q)


def adjust_ws_gs_comp(ws, bms, gs):
    """Adjusts the compatibility of widths and groups."""
    ws_bot = [int(w * b) for w, b in zip(ws, bms)]
    gs = [min(g, w_bot) for g, w_bot in zip(gs, ws_bot)]
    ws_bot = [quantize_float(w_bot, g) for w_bot, g in zip(ws_bot, gs)]
    ws = [int(w_bot / b) for w_bot, b in zip(ws_bot, bms)]
    return ws, gs


def get_stages_from_blocks(ws, rs):
    """Gets ws/ds of network at each stage from per block values."""
    ts_temp = zip(ws + [0], [0] + ws, rs + [0], [0] + rs)
    ts = [w != wp or r != rp for w, wp, r, rp in ts_temp]
    s_ws = [w for w, t in zip(ws, ts[:-1]) if t]
    s_ds = np.diff([d for d, t in zip(range(len(ts)), ts) if t]).tolist()
    return s_ws, s_ds


def generate_regnet(w_a, w_0, w_m, d, q=8):
    """Generates per block ws from RegNet parameters."""
    assert w_a >= 0 and w_0 > 0 and w_m > 1 and w_0 % q == 0
    ws_cont = np.arange(d) * w_a + w_0
    ks = np.round(np.log(ws_cont / w_0) / np.log(w_m))
    ws = w_0 * np.power(w_m, ks)
    ws = np.round(np.divide(ws, q)) * q
    num_stages, max_stage = len(np.unique(ws)), ks.max() + 1
    ws, ws_cont = ws.astype(int).tolist(), ws_cont.tolist()
    return ws, num_stages, max_stage, ws_cont


class RegNet(AnyNet):
    """RegNet model."""

    @staticmethod
    def get_args(cfg):
        """Convert RegNet to AnyNet parameter format."""
        # Generate RegNet ws per block
        w_a, w_0, w_m, d = cfg.REGNET.WA, cfg.REGNET.W0, cfg.REGNET.WM, cfg.REGNET.DEPTH
        ws, num_stages, _, _ = generate_regnet(w_a, w_0, w_m, d)
        # Convert to per stage format
        s_ws, s_ds = get_stages_from_blocks(ws, ws)
        # Use the same gw, bm and ss for each stage
        s_gs = [cfg.REGNET.GROUP_W for _ in range(num_stages)]
        s_bs = [cfg.REGNET.BOT_MUL for _ in range(num_stages)]
        s_ss = [cfg.REGNET.STRIDE for _ in range(num_stages)]
        # Adjust the compatibility of ws and gws
        s_ws, s_gs = adjust_ws_gs_comp(s_ws, s_bs, s_gs)
        # Get AnyNet arguments defining the RegNet
        return {
            "stem_type": cfg.REGNET.STEM_TYPE,
            "stem_w": cfg.REGNET.STEM_W,
            "block_type": cfg.REGNET.BLOCK_TYPE,
            "ds": s_ds,
            "ws": s_ws,
            "ss": s_ss,
            "bms": s_bs,
            "gws": s_gs,
            "se_r": cfg.REGNET.SE_R if cfg.REGNET.SE_ON else None,
            "nc": cfg.CLASS_NUM,
        }

    def __init__(self, cfg=None, logger=None):
        kwargs = RegNet.get_args(cfg)
        super(RegNet, self).__init__(cfg, logger, **kwargs)

    # @staticmethod
    # def complexity(cx, **kwargs):
    #     """Computes model complexity. If you alter the model, make sure to update."""
    #     kwargs = RegNet.get_args() if not kwargs else kwargs
    #     return AnyNet.complexity(cx, **kwargs)

In [5]:



import torch
import torch.nn as nn
import torch.nn.functional as F


__all__ = ['ResNet18','ResNet34','ResNet50','ResNet101','ResNet152']


class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self,in_dim,out_dim,stride=1,op="A"):
        super(BasicBlock,self).__init__()
        self.subconv_1 = nn.Sequential(
            nn.Conv2d(in_dim,out_dim,3,stride,1,bias=False),
            nn.BatchNorm2d(out_dim),
            nn.ReLU(inplace=True),)
        self.subconv_2 = nn.Sequential(
            nn.Conv2d(out_dim,out_dim,3,1,1,bias=False),
            nn.BatchNorm2d(out_dim))
        if in_dim == out_dim and stride == 1:
            self.downsample = nn.Sequential()
        elif op == 'A':
            self.downsample =LambdaLayer(lambda x: F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, out_dim//4, out_dim//4), "constant", 0))
        elif op == 'B':
            self.downsample = nn.Sequential(
                nn.Conv2d(in_dim,out_dim,1,stride,0,bias=False),
                nn.BatchNorm2d(out_dim),
            )
        else: raise ValueError
 
    def forward(self,input_):
        x_0 = self.subconv_1(input_)
        x_1 = self.subconv_2(x_0)
        x_input = self.downsample(input_) 
        x_final = F.relu(x_input + x_1,inplace=True)
        return x_final


class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self,in_dim,out_dim,stride=1,op="B"):
        super(BottleNeck,self).__init__()
        self.subconv_1 = nn.Sequential(
            nn.Conv2d(in_dim,int(out_dim/self.expansion),1,stride,0,bias=False),
            nn.BatchNorm2d(int(out_dim/self.expansion)),
            nn.ReLU(inplace=True),)
        self.subconv_2 = nn.Sequential(
            nn.Conv2d(int(out_dim/self.expansion),
                      int(out_dim/self.expansion),3,1,1,bias=False),
            nn.BatchNorm2d(int(out_dim/self.expansion)),
            nn.ReLU(inplace=True),)
        self.subconv_3 = nn.Sequential(
            nn.Conv2d(int(out_dim/self.expansion),out_dim,1,1,0,bias=False),
            nn.BatchNorm2d(out_dim),)
        if in_dim == out_dim and stride == 1:
            self.downsample = nn.Sequential()
        else:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_dim,out_dim,1,stride,0,bias=False),
                nn.BatchNorm2d(out_dim),
            )

    def forward(self,input_):
        x_0 = self.subconv_1(input_)
        x_1 = self.subconv_2(x_0)
        x_2 = self.subconv_3(x_1)
        x_input = self.downsample(input_)
        x_final = F.relu(x_input+x_2,inplace=True)
        return x_final
    

class ResNet(nn.Module):
    def __init__(self, cfg, logger):
        '''
        block, BLOCK_LIST, in_dim, 
        class_num, BASE=64, use_fc=True, CONV1=(7,2,3),
        MAX_POOL=True, pretrained=False
        '''
        super(ResNet,self).__init__()
        self.head_conv = nn.Sequential(
            nn.Conv2d(cfg.IN_DIM,cfg.BASE,cfg.CONV1[0],cfg.CONV1[1],cfg.CONV1[2],bias=False),
            nn.BatchNorm2d(cfg.BASE),
            nn.ReLU(inplace=True),)
        if cfg.MAX_POOL:
            self.maxpool_1 = nn.MaxPool2d(3,2,1)
        else:
            self.maxpool_1 = nn.Sequential()
        block = BottleNeck if cfg.BLOCK == 'bottleneck' else BasicBlock
        b_ = block.expansion
        self.layer_1 = self._make_layer(block,cfg.BASE,cfg.BASE*b_,cfg.BLOCK_LIST[0],cfg.STRIDE1,cfg.OPERATION)
        self.layer_2 = self._make_layer(block,cfg.BASE*b_,cfg.BASE*2*b_,cfg.BLOCK_LIST[1],2,cfg.OPERATION)
        self.layer_3 = self._make_layer(block,cfg.BASE*2*b_,cfg.BASE*4*b_,cfg.BLOCK_LIST[2],2,cfg.OPERATION)
        self.layer_4 = self._make_layer(block,cfg.BASE*4*b_,cfg.BASE*8*b_,cfg.BLOCK_LIST[3],2,cfg.OPERATION)

        final_feature = cfg.BASE*4*b_ if cfg.BLOCK_LIST[3] == 0 else cfg.BASE*8*b_
        if cfg.USE_FC:
            self.avgpool_1 = nn.AdaptiveAvgPool2d((1,1))
            self.fc_1 = nn.Sequential(
                nn.Flatten(),
                nn.Linear(final_feature,cfg.CLASS_NUM),)
        else:
            self.avgpool_1 = nn.Sequential()
            self.fc_1 = nn.Sequential()
        if cfg.DROPOUT > 0: 
            self.dropout = nn.Dropout(p=cfg.DROPOUT)
        else:
            self.dropout = nn.Sequential()
        self.logger = logger
        self.pretrained = cfg.PRETRAINED
        self._initialization()
    
    def _initialization(self):
        if self.pretrained is not False:
            self.modules.load_state_dict(model_zoo.load_url(model_urls[self.pretrained]))
           
        else:
            for name, sub_module in self.named_modules():
                if isinstance(sub_module, nn.Conv2d) or isinstance(sub_module, nn.ConvTranspose2d) or \
                    isinstance(sub_module, nn.Linear):
                    nn.init.kaiming_normal_(sub_module.weight)
                    # nn.init.kaiming_normal_(sub_module.weight,mode='fan_out'
                    #                         ,nonlinearity='relu')
                    if self.logger is not None:
                        self.logger.info('init {}.weight as kaiming_normal_'.format(name))
                    if sub_module.bias is not None:
                        nn.init.constant_(sub_module.bias, 0.0)
                        if self.logger is not None:
                            self.logger.info('init {}.bias as 0'.format(name))
                # elif isinstance(sub_module, nn.BatchNorm2d):
                #     nn.init.constant_(sub_module.weight,1)
                #     nn.init.constant_(sub_module.bias,0)
                #     if self.logger is not None:
                #         self.logger.info('init {}.weight as constant_ 1'.format(name))
                #         self.logger.info('init {}.bias as constant_ 0'.format(name))
            
    def _make_layer(self,block,in_dim,out_dim,layer_num,stride,op):
        net_layers = []
        if layer_num == 0:
            return nn.Sequential()
        else:    
            for layer in range(layer_num):
                if layer == 0:
                    net_layers.append(block(in_dim,out_dim,stride,op))
                else:
                    net_layers.append(block(out_dim,out_dim,1,op))
            return nn.Sequential(*net_layers)
                    
    def forward(self,input_):
        x = self.head_conv(input_)
        x = self.maxpool_1(x)
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = self.avgpool_1(x)
        x = self.dropout(x)
        x = self.fc_1(x)
        return x       

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

def ResNet18(pretrained = False):
    model = ResNet(BasicBlock, [2, 2, 2, 2])
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model

def ResNet34(pretrained = False):
    model = ResNet(BasicBlock, [3, 4, 6, 3])
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model

def ResNet50(pretrained = False):
    model = ResNet(BottleNeck, [3, 4, 6, 3])
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model

def ResNet101(pretrained = False):
    model = ResNet(BottleNeck, [3, 4, 23, 3])
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model

def ResNet152(pretrained = False):
    model = ResNet(BottleNeck, [3, 8, 36, 3])
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F


__all__ = ['ResNeXt50']


class BasicBlock(nn.Module):
    expansion = 2
    def __init__(self,in_dim,out_dim,stride=1,cardinality=32):
        super(BasicBlock,self).__init__()
        self.layers_ = self._make_layers(in_dim,out_dim,stride,cardinality)
        if in_dim == out_dim and stride == 1:
            self.downsample = None
        else:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_dim,out_dim,1,stride,0,bias=False),
                nn.BatchNorm2d(out_dim),
            )

    def _make_layers(self,in_dim,out_dim,stride,cardinality):
        layers = []
        for group in range(cardinality):
            layers.append(nn.Sequential(
                nn.BatchNorm2d(in_dim),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_dim,out_dim//self.expansion//cardinality,1,stride,0,bias=False),
                nn.BatchNorm2d(out_dim//self.expansion//cardinality),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_dim//self.expansion//cardinality,
                out_dim//self.expansion//cardinality,3,1,1,bias=False),
                nn.BatchNorm2d(out_dim//self.expansion//cardinality),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_dim//self.expansion//cardinality,out_dim,1,1,0,bias=False),
            ))
        return nn.Sequential(*layers)

    def forward(self,input_):
        x_input = input_
        if self.downsample is not None:
            x_input = self.downsample(input_) 
        x_sum = torch.zeros_like(x_input)
        for group in self.layers_:
            x_ = group(input_)
            x_sum += x_   
        x_final = x_input + x_sum
        return x_final


class BasicBlock_pytorch_group(nn.Module):
    expansion = 2
    def __init__(self,in_dim,out_dim,stride=1,cardinality=32):
        super(BasicBlock_pytorch_group,self).__init__()
        self.layers_ = nn.Sequential(
                nn.BatchNorm2d(in_dim),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_dim,out_dim//self.expansion,1,stride,0,bias=False),
                nn.BatchNorm2d(out_dim//self.expansion),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_dim//self.expansion,
                out_dim//self.expansion,3,1,1,bias=False,groups = cardinality),
                nn.BatchNorm2d(out_dim//self.expansion),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_dim//self.expansion,out_dim,1,1,0,bias=False),
            )
        if in_dim == out_dim and stride == 1:
            self.downsample = None
        else:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_dim,out_dim,1,stride,0,bias=False),
                nn.BatchNorm2d(out_dim),
            )

    def forward(self,input_):
        x_input = input_
        if self.downsample is not None:
            x_input = self.downsample(input_) 
        x_ =  self.layers_(input_)
        x_final = x_input + x_
        return x_final


class ResNet(nn.Module):
    def __init__(self,block,block_list,cardinality):
        super(ResNet,self).__init__()
        self.head_conv = nn.Sequential(
            nn.Conv2d(3,64,7,2,3,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),)
        self.maxpool_1 = nn.MaxPool2d(3,2,1)
        b_ = block.expansion
        self.layer_1 = self._make_layer(block,64,128*b_,block_list[0],1,cardinality)
        self.layer_2 = self._make_layer(block,128*b_,256*b_,block_list[1],2,cardinality)
        self.layer_3 = self._make_layer(block,256*b_,512*b_,block_list[2],2,cardinality)
        self.layer_4 = self._make_layer(block,512*b_,1024*b_,block_list[3],2,cardinality)
        self.avgpool_1 = nn.AdaptiveAvgPool2d((1,1))
        self.fc_1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024*b_,1000),
            nn.Softmax(dim = 1),)
        self._initialization()
    
    def _initialization(self):
        for m in self.modules():
            if isinstance (m,nn.Conv2d):
                nn.init.kaiming_normal_(m.weight,mode='fan_out'
                                        ,nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)
    
    def _make_layer(self,block,in_dim,out_dim,layer_num,stride,cardinality):
        net_layers = []
        for layer in range(layer_num):
            if layer == 0:
                net_layers.append(block(in_dim,out_dim,stride,cardinality))
            else:
                net_layers.append(block(out_dim,out_dim,1,cardinality))
        return nn.Sequential(*net_layers)
                    
    def forward(self,input_):
        x = self.head_conv(input_)
        x = self.maxpool_1(x)
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = self.avgpool_1(x)
        x = self.fc_1(x)
        return x       


def ResNeXt50(pretrained = False):
    model = ResNet(BasicBlock_pytorch_group, [3, 4, 6, 3],32)
    return model

    
def _test():
    from torchsummary import summary
    model = ResNeXt50()
    model = model.cuda()
    summary(model,input_size=(3,224,224))


In [7]:
# from backbone import ResNet2015
# from backbone import RegNet2020
# from backbone import effnet

# NET_LUT = {
#         'resnet': ResNet2015.ResNet,
#         'regnet': RegNet2020.RegNet,
#         'resnext': RegNet2020.AnyNet,
#         'effnet': effnet.EffNet,
#     }

def load_regnet_weight(model,pretrain_path,sub_name):
    from collections import OrderedDict
    import torch
    checkpoints = torch.load(pretrain_path+WEIGHT_LUT[sub_name])
    states_no_module = OrderedDict()
    for k, v in checkpoints['model_state'].items():
        if k != 'head.fc.weight' and k!= 'head.fc.bias':
            name_no_module = k
            states_no_module[name_no_module] = v
    model.load_state_dict(states_no_module,strict=False)


# LOAD_LUT = {
#         'resnet': ResNet2015.ResNet,
#         'regnet': load_regnet_weight,
#         'resnext': load_regnet_weight,
#         'effnet': load_regnet_weight,
#     }

# WEIGHT_LUT = {
#         'RegNetY-8.0GF': 'regnet/RegNetY-8.0GF_dds_8gpu.pyth',
#         'RegNetX-4.0GF': 'regnet/RegNetX-4.0GF_dds_8gpu.pyth',
#         'RegNetY-32GF': 'regnet/RegNetY-32GF_dds_8gpu.pyth',
#         'ResNeXt-50': 'resnext/X-50-32x4d_dds_8gpu.pyth',
#         'EfficientNet-B2': 'effnet/EN-B2_dds_8gpu.pyth',
#     }


def get_network(net_name, logger=None, cfg=None):
    try:
        net_class = NET_LUT.get(net_name)
    except:
        logger.error("network tpye error, {} not exist".format(net_name))
    net_instance = net_class(cfg=cfg, logger=logger)
    if cfg.PRETRAIN is not None:
        load_func = LOAD_LUT.get(net_name)
        load_func(net_instance,cfg.PRETRAIN_PATH,cfg.PRETRAIN)
        logger.info("load {} pretrain weight success".format(net_name))
    return net_instance
    

# if __name__ == "__main__":
#     import logging
#     from config import cfg, load_cfg
#     from ptflops import get_model_complexity_info
#     logger = load_cfg() 
#     model = get_network('resnet', logger=logger, cfg=cfg.MODEL)
#     model = model.cuda()
#     flops, params = get_model_complexity_info(model,  (3, 224, 224), 
#         as_strings=True, print_per_layer_stat=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', flops))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params))


In [8]:
def mixup_data(x, t, alpha=1.0):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    t_a, t_b = t, t[index]
    return mixed_x, t_a, t_b, lam


In [9]:
import torch

class WarmLR(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, lr_lambda, last_epoch=-1):
        self.optimizer = optimizer

        if not isinstance(lr_lambda, list) and not isinstance(lr_lambda, tuple):
            self.lr_lambdas = [lr_lambda] * len(optimizer.param_groups)
        else:
            if len(lr_lambda) != len(optimizer.param_groups):
                raise ValueError("Expected {} lr_lambdas, but got {}".format(
                    len(optimizer.param_groups), len(lr_lambda)))
            self.lr_lambdas = list(lr_lambda)
        self.last_epoch = last_epoch
        super(WarmLR, self).__init__(optimizer, last_epoch)


    def state_dict(self):
        import types
        state_dict = {key: value for key, value in self.__dict__.items() if key not in ('optimizer', 'lr_lambdas')}
        state_dict['lr_lambdas'] = [None] * len(self.lr_lambdas)

        for idx, fn in enumerate(self.lr_lambdas):
            if not isinstance(fn, types.FunctionType):
                state_dict['lr_lambdas'][idx] = fn.__dict__.copy()

        return state_dict


    def load_state_dict(self, state_dict):
        lr_lambdas = state_dict.pop('lr_lambdas')
        self.__dict__.update(state_dict)
        state_dict['lr_lambdas'] = lr_lambdas

        for idx, fn in enumerate(lr_lambdas):
            if fn is not None:
                self.lr_lambdas[idx].__dict__.update(fn)
       

    def get_lr(self):
        if self.last_epoch > 0:
            return [group['lr'] + lmbda(self.last_epoch)
                    for lmbda, group in zip(self.lr_lambdas, self.optimizer.param_groups)]
        else:
            return list(self.base_lrs)


def get_opt(model, cfg_train, logger=None, is_warm=False, its_total=0):
    trainable_vars = [param for param in model.parameters() if param.requires_grad]
    if is_warm:
        opt = torch.optim.SGD(trainable_vars, 
            lr=cfg_train.LR_WARM, 
            momentum=cfg_train.BETA1,
            weight_decay=cfg_train.WEIGHT_DECAY,)
        factor = float((cfg_train.LR) / its_total)
        lmbda = lambda its: factor
        lr_scheduler = WarmLR(opt, lmbda, last_epoch=-1)
        return opt, lr_scheduler

    if cfg_train.TYPE == 'adamw':
        opt = torch.optim.AdamW(trainable_vars, 
            lr=cfg_train.LR, 
            betas=(cfg_train.BETA1, cfg_train.BETA2),
            eps=1e-08, 
            weight_decay=cfg_train.WEIGHT_DECAY,
            amsgrad=False)
    elif cfg_train.TYPE == 'sgd':
        opt = torch.optim.SGD(trainable_vars, 
            lr=cfg_train.LR, 
            momentum=cfg_train.BETA1,
            weight_decay=cfg_train.WEIGHT_DECAY,)
    else:
        logger.error("{} not exist in opt type".format(cfg_train.TYPE)) 

    if cfg_train.LR_TYPE == 'cos':
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, its_total, eta_min=0, last_epoch=-1)
    else:
        lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(opt,cfg_train.LR_REDUCE,
                                                        gamma=cfg_train.LR_FACTOR,
                                                        last_epoch=-1)
    return opt,lr_scheduler

In [10]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [11]:
#!usr/bin/env python3
#-*- coding=utf-8 -*-
#python=3.6 pytorch=1.2.0


import os
import random
import cv2
import numpy as np
import torch
import torch.nn.functional as F
from tensorboardX import SummaryWriter
#from easydict import EasyDict as edict
from random import randint
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import roc_auc_score
import time

# from network_factory import get_network
# from opt_factory import get_opt
# from datasets.loader_factory import get_loader
# from test import val
# from utils import AUC, AverageMeter,load_cfg,plot_result_data,\
#     load_checkpoints,print_to_screen,save_checkpoints



def train(cfg):
    logger = load_cfg(cfg)    
   
    train_loader = get_loader(cfg.DATASET_TRPE, cfg.PATH.DATA, 'train', label_path=cfg.PATH.LABEL, cfg=cfg.TRAIN, logger=logger)
    val_loader = get_loader(cfg.DATASET_TRPE, cfg.PATH.EVAL, 'eval',label_path=cfg.PATH.LABEL, cfg=cfg.TRAIN, logger=logger)
    its_num = len(train_loader)
    # from torchvision import models
    # import torch.nn as nn
    # class ResNet(nn.Module):
    #     def __init__(self, pre_trained=True, n_class=200, model_choice=50):
    #         super(ResNet, self).__init__()
    #         self.n_class = n_class
    #         self.base_model = self._model_choice(pre_trained, model_choice)
    #         self.base_model.avgpool = nn.AdaptiveAvgPool2d((1,1))
    #         self.base_model.fc = nn.Linear(512*4, n_class)
    #         #self.base_model.fc.apply(weight_init_kaiming)

    #     def forward(self, x):
    #         N = x.size(0)
    #         assert x.size() == (N, 3, 224, 224)
    #         x = self.base_model(x)
    #         assert x.size() == (N, self.n_class)
    #         return x

    #     def _model_choice(self, pre_trained, model_choice):
    #         if model_choice == 50:
    #             return models.resnet50(pretrained=pre_trained)
    #         elif model_choice == 101:
    #             return models.resnet101(pretrained=pre_trained)
    #         elif model_choice == 152:
    #             return models.resnet152(pretrained=pre_trained)

    model = get_network(cfg.MODEL.NAME, cfg=cfg.MODEL, logger=logger)
    model = torch.nn.DataParallel(model).cuda() 
   

    cfg.TRAIN.LR_REDUCE = [int(its_num*x) for x in cfg.TRAIN.LR_REDUCE]
    opt_t,lr_scheduler_t = get_opt(model, cfg.TRAIN, logger, 
        its_total=(cfg.TRAIN.EPOCHS-cfg.TRAIN.WARMUP)*its_num)
    if cfg.TRAIN.WARMUP != 0:
        warm_opt, warm_scheduler = get_opt(model, cfg.TRAIN, logger, 
            is_warm=True, its_total=cfg.TRAIN.WARMUP*its_num)

    loss_func = torch.nn.BCEWithLogitsLoss()
    current_epoch = load_checkpoints(model, opt_t, cfg.PATH , logger, lr_scheduler_t)
    log_writter = SummaryWriter(cfg.PATH.EXPS+cfg.PATH.NAME)
    
    auc_total = []
    auc_val_total = []
    loss_total = []
    losss_val_total = []
    best_val = [0,0]
   
    for epoch in range(current_epoch, cfg.TRAIN.EPOCHS):
        start_time = time.time()
        if epoch < cfg.TRAIN.WARMUP:
            opt = warm_opt
            lr_scheduler = warm_scheduler
        else:
            opt = opt_t
            lr_scheduler = lr_scheduler_t

        auc_train_class = AUC()
        loss_train_calss = AverageMeter()
        model.train()
        data_begin = time.time()
        for its, (imgs, targets)in enumerate(train_loader):
            data_time = time.time()-data_begin
            imgs = imgs.cuda() if torch.cuda.is_available() else imgs
            targets = targets.cuda() if torch.cuda.is_available() else targets
            
                      
            opt.zero_grad()
            imgs, ta, tb, lam = mixup_data(imgs, targets.view(-1, 1))
            outputs = model(imgs)
            loss = lam * loss_func(outputs,ta) + (1-lam) * loss_func(outputs,tb)
            loss.backward()
            opt.step()
            lr_scheduler.step()
            
            loss_train_calss.add_value(loss.cpu())
            train_time = time.time()-(data_time+data_begin)
            data_begin = time.time()
            lr = opt.param_groups[0]['lr']
            mem = torch.cuda.memory_cached() / 1E9 if torch.cuda.is_available() else 0
#             try: 
#                 auc_train_class.add_value(roc_auc_score(targets.cpu().numpy(), outputs.cpu().numpy())
#             except:
#                 auc_train_class.add_value(0.50)                          
            if its % cfg.PRINT_FRE == 0:
                print_to_screen(loss, lr, its, epoch, its_num, logger, 
                    data_time,train_time,mem,auc_train_class.print_())
            
            # if cfg.SHORT_TEST == True:
            #     if its == 20:
            #         break

        save_checkpoints(cfg.PATH.EXPS+cfg.PATH.NAME+cfg.PATH.MODEL, model, opt, epoch,lr_scheduler)
        auc_val, loss_val = val(val_loader, model, logger, loss_func, epoch, print_fre=cfg.PRINT_FRE,)
        log_writter.add_scalars("auc",{'auc_train':auc_train_class.print_(),
                                     'auc_val':auc_val,},
                                     epoch)
        auc_total.append(auc_train_class.print_())
        auc_val_total.append(auc_val)
        loss_total.append(loss_train_calss.avg())
        losss_val_total.append(loss_val)
        end_time = time.time()-start_time
        logger.info('Train Auc@1:%.4f\t'%(auc_train_class.print_())+'Val Auc@1:%.4f\t'%(auc_val)+'Epoch Time:%.2fmin'%(end_time/60))
        if best_val[0] < auc_val:
            best_val[0] = auc_val
            best_val[1] = epoch
            save_checkpoints(cfg.PATH.EXPS+cfg.PATH.NAME+cfg.PATH.BESTMODEL, model, opt, epoch,lr_scheduler)
        logger.info('BestV Auc@1:%.4f\t'%(best_val[0])+"Best Epoch:%d"%(best_val[1]))

    plot_result_data(auc_total,auc_val_total,loss_total,
        losss_val_total,cfg.PATH.EXPS+cfg.PATH.NAME, cfg.TRAIN.EPOCHS)
    log_writter.close()


# if __name__ == "__main__":
#     from config import cfg
#     train(cfg)         

In [12]:
import os 
import torch 
import torch.nn as nn
import numpy as np
#from easydict import EasyDict as edict
import logging
import cv2
import time
import pandas
import csv

# from network_factory import get_network
# from datasets.loader_factory import get_loader
# from utils import load_test_checkpoints, AUC, \
#     AverageMeter, load_cfg, print_to_screen

def embedding(test_loader, model_0, model_1, model_2, logger, print_fre=50):
    model_0.eval()
    model_1.eval()
    #model_2.eval()
    its_num = len(val_loader)
    outputs_all = []
    ids_all = []
    data_begin = time.time()
    with torch.no_grad():
        for its, (ids, imgs) in enumerate(test_loader):
            data_time = time.time()-data_begin
            imgs = imgs.cuda() 
            outputs_0 = model_0(imgs)
            outputs_1 = model_1(imgs)
            #outputs_2 = model_2(imgs)
            #outputs = (outputs_0.sigmoid() + outputs_1.sigmoid() + outputs_2.sigmoid())/3
            outputs = (outputs_0.sigmoid() + outputs_1.sigmoid())/2
            loss = 0
            train_time = time.time()-(data_time+data_begin)
            data_begin = time.time()
            mem = torch.cuda.memory_cached() / 1E9 if torch.cuda.is_available() else 0
            if its % print_fre == 0:
                print_to_screen(loss, 0, its, 0, its_num,
                    logger, data_time, train_time, mem)  
            outputs_all.append(outputs.cpu().numpy())
            ids_all.append(ids)
    ids_all = np.concatenate(ids_all)        
    outputs_all = np.concatenate(outputs_all)
    with open ("./new_embedding.csv", "w") as f:
        writer = csv.writer(f, delimiter=' ')
        writer.writerow(["ids", "targets"])
        for ids, outputs in zip(ids_all, outputs_all):
            writer.writerow([ids, outputs])
    


def val(val_loader, model, logger=None, loss_function=None, epoch=0, print_fre=50):  
    model.eval()
    its_num = len(val_loader)
    auc_single_val = AverageMeter()
    loss_val = AverageMeter()
    data_begin = time.time()
    with torch.no_grad():
        for its, (imgs, targets) in enumerate(val_loader):
            data_time = time.time()-data_begin
            imgs = imgs.cuda() if torch.cuda.is_available() else imgs
            targets = targets.cuda() if torch.cuda.is_available() else targets
            outputs = model(imgs)
            loss = loss_function(outputs,targets) if loss_function is not None else torch.tensor(0)
            train_time = time.time()-(data_time+data_begin)
            data_begin = time.time()
            loss_val.add_value(loss.cpu())
#             try: 
#                 auc_single_val.add_value(roc_auc_score(targets.cpu().numpy(), outputs.cpu().numpy())
#             except:
#                 auc_single_val.add_value(0.50)  
            mem = torch.cuda.memory_cached() / 1E9 if torch.cuda.is_available() else 0
            if its % print_fre == 0:
                print_to_screen(loss, 0, its, epoch, its_num,
                    logger, data_time, train_time, mem, acc=acc_single_val.print_())               
    return auc_single_val.print_(), loss_val.avg()        
        

# if __name__ == "__main__":
#     import sys
#     if len(sys.argv) < 2:
#         print ("Usage: python test.py [eval, test, embedding]")
#     else:
#         cmd = sys.argv[1]
#         from config_cub import cfg
#         logger = load_cfg(cfg)
#         model = get_network(cfg.MODEL.NAME, cfg=cfg.MODEL, logger=logger)
#         model = torch.nn.DataParallel(model, cfg.GPUS).cuda() if torch.cuda.is_available() else model
#         load_test_checkpoints(model, cfg.PATH, logger, use_best=True)
        
#         if cmd == 'eval':   
#             test_loader = get_loader(cfg.DATASET_TRPE, cfg.PATH.EVAL, 'eval',label_path=cfg.PATH.LABEL, cfg=cfg.TRAIN, logger=logger)
#             acc_val, _ = val(test_loader, model, logger, print_fre=cfg.PRINT_FRE,)
#             logger.info("Prec@1:%.4f"%(acc_val))

#         elif cmd =='test':
#             test_loader = get_loader(cfg.DATASET_TRPE, cfg.PATH.TEST, 'test', cfg.TRAIN, logger)
#             test(test_loader, model, logger)

#         elif cmd =='embedding':
#             test_loader = get_loader(cfg.DATASET_TRPE, cfg.PATH.EVAL, 'eval',label_path=cfg.PATH.LABEL, cfg=cfg.TRAIN, logger=logger)
#             from config_cub_1 import cfg1
#             logger = load_cfg(cfg1)
#             model_1 = get_network(cfg1.MODEL.NAME, cfg=cfg1.MODEL, logger=logger)
#             model_1 = torch.nn.DataParallel(model_1, cfg1.GPUS).cuda() if torch.cuda.is_available() else model_1
#             load_test_checkpoints(model_1, cfg1.PATH, logger, use_best=True)
#             test_loader_1 = get_loader(cfg1.DATASET_TRPE, cfg1.PATH.EVAL, 'eval',label_path=cfg1.PATH.LABEL, cfg=cfg1.TRAIN, logger=logger)
#             from config_cub_2 import cfg2
#             logger = load_cfg(cfg2)
#             model_2 = get_network(cfg2.MODEL.NAME, cfg=cfg2.MODEL, logger=logger)
#             model_2 = torch.nn.DataParallel(model_2, cfg2.GPUS).cuda() if torch.cuda.is_available() else model_2
#             load_test_checkpoints(model_2, cfg2.PATH, logger, use_best=True)
#             acc_val = embedding(test_loader, test_loader_1, model,model_1,model_2, logger)
#             logger.info("Embedding Prec@1:%.4f"%(acc_val))
#         else:
#             print ("Usage: python test.py [eval, test]")

In [13]:
# import os
# from easydict import EasyDict as edict


# cfg1 = edict()
# cfg1.PATH = edict()
# cfg1.PATH.DATA =  ['./waixingren/']
# cfg1.PATH.LABEL = './waixingren/train_label.csv'
# cfg1.PATH.EVAL = ['/waixingren/']
# cfg1.PATH.TEST = './waixingren/sample_submission.csv'
# cfg1.PATH.RES_TEST = './res_imgs/'
# cfg1.PATH.EXPS = './exps/'
# cfg1.PATH.NAME = 'seti_effnet'
# cfg1.PATH.MODEL = '/model.pth'
# cfg1.PATH.BESTMODEL = '/bestmodel.pth'
# cfg1.PATH.LOG = '/log.txt'
# cfg1.PATH.RESULTS = '/results/'


# cfg1.DETERMINISTIC = edict()
# cfg1.DETERMINISTIC.SEED = 60
# cfg1.DETERMINISTIC.CUDNN = True


# cfg1.TRAIN = edict()
# cfg1.TRAIN.EPOCHS = 20
# cfg1.TRAIN.BATCHSIZE = 32
# cfg1.TRAIN.L1SCALING = 100
# cfg1.TRAIN.TYPE = 'adamw'
# cfg1.TRAIN.LR = 1e-3
# cfg1.TRAIN.BETA1 = 0.9
# cfg1.TRAIN.BETA2 = 0.999
# cfg1.TRAIN.LR_TYPE = 'cos'
# cfg1.TRAIN.LR_REDUCE = [26,36]
# cfg1.TRAIN.LR_FACTOR = 0.1
# cfg1.TRAIN.WEIGHT_DECAY = 1e-4
# cfg1.TRAIN.NUM_WORKERS = 16
# cfg1.TRAIN.WARMUP = 0
# cfg1.TRAIN.LR_WARM = 1e-7
# #-------- data aug --------#
# cfg1.TRAIN.USE_AUG = True
# cfg1.TRAIN.CROP = 620
# cfg1.TRAIN.PAD = 0
# cfg1.TRAIN.RESIZE = 620
# cfg1.TRAIN.ROATION = 45


# cfg1.MODEL = edict()
# cfg1.MODEL.NAME = 'efficienet'
# cfg1.MODEL.IN_DIM = 1
# cfg1.MODEL.CLASS_NUM = 1 
# cfg1.MODEL.USE_FC = True
# cfg1.MODEL.PRETRAIN = True
# cfg1.MODEL.PRETRAIN_PATH = './exps/pretrain/'
# cfg1.MODEL.DROPOUT = 0.5
# cfg1.MODEL.LOSS = 'bce_only_g' 
# #-------- for resnet --------#
# cfg1.MODEL.BLOCK = 'bottleneck'
# cfg1.MODEL.BLOCK_LIST = [3,4,6,3] 
# cfg1.MODEL.CONV1 = (7,2,3)
# cfg1.MODEL.OPERATION = 'B'
# cfg1.MODEL.STRIDE1 = 1
# cfg1.MODEL.MAX_POOL = True
# cfg1.MODEL.BASE = 64
# #-------- for regnet --------#
# cfg1.MODEL.REGNET = edict()
# cfg1.MODEL.REGNET.STEM_TYPE = "simple_stem_in"
# cfg1.MODEL.REGNET.STEM_W = 32
# cfg1.MODEL.REGNET.BLOCK_TYPE = "res_bottleneck_block"
# cfg1.MODEL.REGNET.STRIDE = 2
# cfg1.MODEL.REGNET.SE_ON = True
# cfg1.MODEL.REGNET.SE_R = 0.25
# cfg1.MODEL.REGNET.BOT_MUL = 1.0
# cfg1.MODEL.REGNET.DEPTH = 20
# cfg1.MODEL.REGNET.W0 = 232
# cfg1.MODEL.REGNET.WA = 115.89
# cfg1.MODEL.REGNET.WM = 2.53
# cfg1.MODEL.REGNET.GROUP_W = 232
# #-------- for anynet -------#
# cfg1.MODEL.ANYNET = edict()
# cfg1.MODEL.ANYNET.STEM_TYPE = "res_stem_in"
# cfg1.MODEL.ANYNET.STEM_W = 64
# cfg1.MODEL.ANYNET.BLOCK_TYPE = "res_bottleneck_block"
# cfg1.MODEL.ANYNET.STRIDES = [1,2,2,2]
# cfg1.MODEL.ANYNET.SE_ON = False
# cfg1.MODEL.ANYNET.SE_R = 0.25
# cfg1.MODEL.ANYNET.BOT_MULS = [0.5,0.5,0.5,0.5]
# cfg1.MODEL.ANYNET.DEPTHS = [3,4,6,3]
# cfg1.MODEL.ANYNET.GROUP_WS = [4,8,16,32]
# cfg1.MODEL.ANYNET.WIDTHS = [256,512,1024,2048]
# #-------- for effnet --------#
# cfg1.MODEL.EFFNET = edict()
# cfg1.MODEL.EFFNET.STEM_W = 32
# cfg1.MODEL.EFFNET.EXP_RATIOS = [1,6,6,6,6,6,6]
# cfg1.MODEL.EFFNET.KERNELS = [3,3,5,3,5,5,3]
# cfg1.MODEL.EFFNET.HEAD_W = 1408
# cfg1.MODEL.EFFNET.DC_RATIO = 0.0
# cfg1.MODEL.EFFNET.STRIDES = [1,2,2,2,1,2,1]
# cfg1.MODEL.EFFNET.SE_R = 0.25
# cfg1.MODEL.EFFNET.DEPTHS = [2, 3, 3, 4, 4, 5, 2]
# cfg1.MODEL.EFFNET.GROUP_WS = [4,8,16,32]
# cfg1.MODEL.EFFNET.WIDTHS = [16,24,48,88,120,208,352]


# cfg1.GPUS = [0]
# cfg1.PRINT_FRE = 300

# cfg1.DATASET_TRPE = 'waixingren'
# cfg1.SHORT_TEST = False


# if __name__ == "__main__":
#     from utils import load_cfg1
#     logger = load_cfg1(cfg1)
#     print(cfg1)






In [14]:
# import os
# from easydict import EasyDict as edict


# cfg2 = edict()
# cfg2.PATH = edict()
# cfg2.PATH.DATA =  ['./waixingren/']
# cfg2.PATH.LABEL = './waixingren/train_label.csv'
# cfg2.PATH.EVAL = ['/waixingren/']
# cfg2.PATH.TEST = './waixingren/sample_submission.csv'
# cfg2.PATH.RES_TEST = './res_imgs/'
# cfg2.PATH.EXPS = './exps/'
# cfg2.PATH.NAME = 'seti_resnext'
# cfg2.PATH.MODEL = '/model.pth'
# cfg2.PATH.BESTMODEL = '/bestmodel.pth'
# cfg2.PATH.LOG = '/log.txt'
# cfg2.PATH.RESULTS = '/results/'


# cfg2.DETERMINISTIC = edict()
# cfg2.DETERMINISTIC.SEED = 60
# cfg2.DETERMINISTIC.CUDNN = True


# cfg2.TRAIN = edict()
# cfg2.TRAIN.EPOCHS = 20
# cfg2.TRAIN.BATCHSIZE = 32
# cfg2.TRAIN.L1SCALING = 100
# cfg2.TRAIN.TYPE = 'adamw'
# cfg2.TRAIN.LR = 1e-3
# cfg2.TRAIN.BETA1 = 0.9
# cfg2.TRAIN.BETA2 = 0.999
# cfg2.TRAIN.LR_TYPE = 'cos'
# cfg2.TRAIN.LR_REDUCE = [26,36]
# cfg2.TRAIN.LR_FACTOR = 0.1
# cfg2.TRAIN.WEIGHT_DECAY = 1e-4
# cfg2.TRAIN.NUM_WORKERS = 16
# cfg2.TRAIN.WARMUP = 0
# cfg2.TRAIN.LR_WARM = 1e-7
# #-------- data aug --------#
# cfg2.TRAIN.USE_AUG = True
# cfg2.TRAIN.CROP = 832
# cfg2.TRAIN.PAD = 0
# cfg2.TRAIN.RESIZE = 832
# cfg2.TRAIN.ROATION = 0



# cfg2.MODEL = edict()
# cfg2.MODEL.NAME = 'resnext'
# cfg2.MODEL.IN_DIM = 1
# cfg2.MODEL.CLASS_NUM = 1 
# cfg2.MODEL.USE_FC = True
# cfg2.MODEL.PRETRAIN = True
# cfg2.MODEL.PRETRAIN_PATH = './exps/pretrain/'
# cfg2.MODEL.DROPOUT = 0
# cfg2.MODEL.LOSS = 'bce_only_g' 
# #-------- for resnet --------#
# cfg2.MODEL.BLOCK = 'bottleneck'
# cfg2.MODEL.BLOCK_LIST = [3,4,6,3] 
# cfg2.MODEL.CONV1 = (7,2,3)
# cfg2.MODEL.OPERATION = 'B'
# cfg2.MODEL.STRIDE1 = 1
# cfg2.MODEL.MAX_POOL = True
# cfg2.MODEL.BASE = 64
# #-------- for regnet --------#
# cfg2.MODEL.REGNET = edict()
# cfg2.MODEL.REGNET.STEM_TYPE = "simple_stem_in"
# cfg2.MODEL.REGNET.STEM_W = 32
# cfg2.MODEL.REGNET.BLOCK_TYPE = "res_bottleneck_block"
# cfg2.MODEL.REGNET.STRIDE = 2
# cfg2.MODEL.REGNET.SE_ON = True
# cfg2.MODEL.REGNET.SE_R = 0.25
# cfg2.MODEL.REGNET.BOT_MUL = 1.0
# cfg2.MODEL.REGNET.DEPTH = 20
# cfg2.MODEL.REGNET.W0 = 232
# cfg2.MODEL.REGNET.WA = 115.89
# cfg2.MODEL.REGNET.WM = 2.53
# cfg2.MODEL.REGNET.GROUP_W = 232
# #-------- for anynet -------#
# cfg2.MODEL.ANYNET = edict()
# cfg2.MODEL.ANYNET.STEM_TYPE = "res_stem_in"
# cfg2.MODEL.ANYNET.STEM_W = 64
# cfg2.MODEL.ANYNET.BLOCK_TYPE = "res_bottleneck_block"
# cfg2.MODEL.ANYNET.STRIDES = [1,2,2,2]
# cfg2.MODEL.ANYNET.SE_ON = False
# cfg2.MODEL.ANYNET.SE_R = 0.25
# cfg2.MODEL.ANYNET.BOT_MULS = [0.5,0.5,0.5,0.5]
# cfg2.MODEL.ANYNET.DEPTHS = [3,4,6,3]
# cfg2.MODEL.ANYNET.GROUP_WS = [4,8,16,32]
# cfg2.MODEL.ANYNET.WIDTHS = [256,512,1024,2048]
# #-------- for effnet --------#
# cfg2.MODEL.EFFNET = edict()
# cfg2.MODEL.EFFNET.STEM_W = 32
# cfg2.MODEL.EFFNET.EXP_RATIOS = [1,6,6,6,6,6,6]
# cfg2.MODEL.EFFNET.KERNELS = [3,3,5,3,5,5,3]
# cfg2.MODEL.EFFNET.HEAD_W = 1408
# cfg2.MODEL.EFFNET.DC_RATIO = 0.0
# cfg2.MODEL.EFFNET.STRIDES = [1,2,2,2,1,2,1]
# cfg2.MODEL.EFFNET.SE_R = 0.25
# cfg2.MODEL.EFFNET.DEPTHS = [2, 3, 3, 4, 4, 5, 2]
# cfg2.MODEL.EFFNET.GROUP_WS = [4,8,16,32]
# cfg2.MODEL.EFFNET.WIDTHS = [16,24,48,88,120,208,352]


# cfg2.GPUS = [0]
# cfg2.PRINT_FRE = 300

# cfg2.DATASET_TRPE = 'waixingren'
# cfg2.SHORT_TEST = False


# if __name__ == "__main__":
#     from utils import load_cfg2
#     logger = load_cfg2(cfg2)
#     print(cfg2)






In [15]:
# the training and test code are above, we train and test the model on ourselves mechines.

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
results = pd.read_csv("/kaggle/input/seti-final/fusion_1.csv")
results.to_csv("submission.csv",index=False)